In [1]:
import ipywidgets
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

plt.style.use('dark_background')
pio.templates.default = 'plotly_dark'

In [4]:
%load_ext autoreload
%autoreload 2
import gapminder
from gapminder.mock import LaserScan

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
N_SAMPLES = 1081

In [6]:
df = pd.read_csv(
    './debug.csv',
    header=None,
    names=[
    # "header",  #
    # "header.seq",  # 5
    # "header.stamp",  #
    "header.stamp.secs",  # 2829
    "header.stamp.nsecs",  #  69000000
    "header.frame_id",  # "laser_sensor_link"
    "angle_min",  # -1.57079994678
    "angle_max",  # 1.57079994678
    "angle_increment",  # 0.00436940183863
    "time_increment",  # 0.0
    "scan_time",  # 0.0
    "range_min",  # 0.10000000149
    "range_max",  # 30.0
    # float[] ranges
    *[f'ranges[{x:>03}]' for x in range(N_SAMPLES)],
    # 'etc',
    None,
    # float[] intensities
    ],
)
df

,header.stamp.secs,header.stamp.nsecs,header.frame_id,angle_min,angle_max,angle_increment,time_increment,scan_time,range_min,range_max,...,ranges[1072],ranges[1073],ranges[1074],ranges[1075],ranges[1076],ranges[1077],ranges[1078],ranges[1079],ranges[1080],NaN
0,1772074964,203558928,laser,-2.356194,2.356194,0.004363,0.000017,0.025,0.02,30.0,...,2.138000,2.136000,2.129,2.126,2.107,2.094,2.085,2.086,2.077,NaN
1,1772074964,233556987,laser,-2.356194,2.356194,0.004363,0.000017,0.025,0.02,30.0,...,2.131000,2.157000,2.157,2.124,2.108,2.090,2.094,2.097,2.081,NaN
2,1772074964,253551735,laser,-2.356194,2.356194,0.004363,0.000017,0.025,0.02,30.0,...,2.137000,2.127000,2.115,2.120,2.109,2.091,2.089,2.091,2.074,NaN
3,1772074964,283494784,laser,-2.356194,2.356194,0.004363,0.000017,0.025,0.02,30.0,...,2.165000,2.123000,2.123,2.110,2.110,2.090,2.088,2.094,2.066,NaN
4,1772074964,303506557,laser,-2.356194,2.356194,0.004363,0.000017,0.025,0.02,30.0,...,2.141000,2.151000,2.141,2.135,2.113,2.076,2.084,2.071,2.059,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,1772074968,913550031,laser,-2.356194,2.356194,0.004363,0.000017,0.025,0.02,30.0,...,2.786000,2.807000,2.823,2.798,2.804,2.788,2.833,2.857,2.866,NaN
189,1772074968,933549738,laser,-2.356194,2.356194,0.004363,0.000017,0.025,0.02,30.0,...,2.792000,2.791000,2.777,2.802,2.830,2.816,2.839,2.866,2.897,NaN
190,1772074968,963485233,laser,-2.356194,2.356194,0.004363,0.000017,0.025,0.02,30.0,...,2.820000,2.814000,2.802,2.833,2.821,2.823,2.849,2.875,2.899,NaN
191,1772074968,983505773,laser,-2.356194,2.356194,0.004363,0.000017,0.025,0.02,30.0,...,65.532997,2.782000,2.782,2.782,2.818,2.827,2.850,2.848,2.851,NaN


In [7]:
msgs = [LaserScan(row) for _, row in df.iterrows()]

In [9]:
node = gapminder.GapMinder()

In [11]:
@ipywidgets.interact
def generate_map(t : ipywidgets.IntSlider(min=0, max=len(msgs) - 1) = 116):
    global msg
    msg = msgs[t]
    r = np.array(msg.ranges)
    r[r > msg.range_max] = np.nan
    r[r < msg.range_min] = np.nan

    vec_r, vec_theta = node.get_drive_vector(msg)

    rtick = node.get_obstructed_distance(msg)
    return go.Figure(
        data=[
            go.Scattergl(
                x=-r * np.sin(node.theta),
                y=r * np.cos(node.theta),
                mode='markers'
            ),
            go.Scattergl(
                x=-rtick * np.sin(node.theta),
                y=rtick * np.cos(node.theta),
                mode='markers',
                marker=dict(
                    size=2,
                    symbol='x',
                ),
            ),
            go.Scattergl(
                x=[0, -vec_r * np.sin(vec_theta)],
                y=[0, vec_r * np.cos(vec_theta)],
                marker=dict(
                    size=10,
                    # symbol= "arrow-bar-up",
                    # angleref="previous",
                ),
            )
        ],
        layout=dict(
            shapes=[
                dict(
                    type="circle",
                    layer="between",
                    xref="x",
                    yref="y",
                    x0=-node.car_width, y0=-node.car_width,
                    x1=node.car_width, y1=node.car_width,
                    line_color="LightSeaGreen",
                ),
            ],
            yaxis=dict(
                scaleanchor='x',
                range=[-1, 3],
            ),
            xaxis=dict(
                range=[-3, 3],
            ),
            width=800,
            height=600,
        ),
    )

interactive(children=(IntSlider(value=116, description='t', max=348, min=-116), Output()), _dom_classes=('widg…

In [13]:
generate_map(116).write_html('./resource/map2d.html')